# exp05

exp03 catboost版

https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977

In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import joblib
import random
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from itertools import combinations

import ipywidgets as widgets
import snappy
# from ipywidgets import interact, Select


In [2]:

# ====================================================
# Get the difference
# ====================================================
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    train = pd.read_parquet('/content/data/train.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()
    test = pd.read_parquet('../input/amex-fe/test_fe.parquet')
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    # Get the difference
    test_diff = get_difference(test, num_featusres)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
    # Save files to disk
    train.to_parquet('../input/amex-fe/train_fe.parquet')
    test.to_parquet('../input/amex-fe/test_fe.parquet')

# Read & Preprocess Data
read_preprocess_data()

FileNotFoundError: [Errno 2] No such file or directory: '/content/data/train.parquet'

# Training & Inference

In [3]:

# # ====================================================
# # Configurations
# # ====================================================
# class CFG:
#     input_dir = '../input/amex-fe/'
#     seed = 45
#     n_folds = 5
#     target = 'target'
#     boosting_type = 'dart'
#     metric = 'binary_logloss'
#     model = "XGB"

# # ====================================================
# # Seed everything
# # ====================================================
# def seed_everything(seed):
#     random.seed(seed)
#     np.random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)

# # ====================================================
# # Read data
# # ====================================================
# def read_data():
#     train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
#     test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
#     return train, test

# # ====================================================
# # XGB train
# # ====================================================

# def cat_train(x, y, xt, yt,cat_features):
#     print("# of features:", x.shape[1])
#     assert x.shape[1] == xt.shape[1]
# #     dtrain = xgb.DMatrix(data=x, label=y)
# #     dvalid = xgb.DMatrix(data=xt, label=yt)

# #     watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
#     watchlist = [(x, 'train'), (xt, 'eval')]
#     clf = CatBoostClassifier(iterations=10, random_state=CFG.seed)# 5000
#     clf.fit(x, y, eval_set=[(xt, yt)], cat_features=cat_features,plot=True,verbose_eval=100)
#     print('best ntree_limit:', bst.best_ntree_limit)
#     print('best score:', bst.best_score)
#     # return clf.predict_proba(xt)[:, 1]
#     return clf.predict_proba(xt)[:, 1], clf



# # ====================================================
# # Amex metric
# # ====================================================
# def amex_metric(y_true, y_pred):
#     labels = np.transpose(np.array([y_true, y_pred]))
#     labels = labels[labels[:, 1].argsort()[::-1]]
#     weights = np.where(labels[:,0]==0, 20, 1)
#     cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
#     top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
#     gini = [0,0]
#     for i in [1,0]:
#         labels = np.transpose(np.array([y_true, y_pred]))
#         labels = labels[labels[:, i].argsort()[::-1]]
#         weight = np.where(labels[:,0]==0, 20, 1)
#         weight_random = np.cumsum(weight / np.sum(weight))
#         total_pos = np.sum(labels[:, 0] *  weight)
#         cum_pos_found = np.cumsum(labels[:, 0] * weight)
#         lorentz = cum_pos_found / total_pos
#         gini[i] = np.sum((lorentz - weight_random) * weight)
#     return 0.5 * (gini[1]/gini[0] + top_four)


# def xgb_amex(y_pred, y_true):
#     return 'amex', amex_metric_np(y_pred,y_true.get_label())


# # Created by https://www.kaggle.com/yunchonggan
# # https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
# def amex_metric_np(preds: np.ndarray, target: np.ndarray) -> float:
#     indices = np.argsort(preds)[::-1]
#     preds, target = preds[indices], target[indices]

#     weight = 20.0 - target * 19.0
#     cum_norm_weight = (weight / weight.sum()).cumsum()
#     four_pct_mask = cum_norm_weight <= 0.04
#     d = np.sum(target[four_pct_mask]) / np.sum(target)

#     weighted_target = target * weight
#     lorentz = (weighted_target / weighted_target.sum()).cumsum()
#     gini = ((lorentz - cum_norm_weight) * weight).sum()

#     n_pos = np.sum(target)
#     n_neg = target.shape[0] - n_pos
#     gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)

#     g = gini / gini_max
#     return 0.5 * (g + d)




# # ====================================================
# # Train & Evaluate
# # ====================================================
# def train_and_evaluate(train, test):
#     # Label encode categorical features
#     cat_features = [
#         "B_30",
#         "B_38",
#         "D_114",
#         "D_116",
#         "D_117",
#         "D_120",
#         "D_126",
#         "D_63",
#         "D_64",
#         "D_66",
#         "D_68"
#     ]
#     cat_features = [f"{cf}_last" for cf in cat_features]
#     for cat_col in cat_features:
#         encoder = LabelEncoder()
#         train[cat_col] = encoder.fit_transform(train[cat_col])
#         test[cat_col] = encoder.transform(test[cat_col])
#     # Round last float features to 2 decimal place
#     num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
#     num_cols = [col for col in num_cols if 'last' in col]
#     for col in num_cols:
#         train[col + '_round2'] = train[col].round(2)
#         test[col + '_round2'] = test[col].round(2)
#     # Get the difference between last and mean
#     num_cols = [col for col in train.columns if 'last' in col]
#     num_cols = [col[:-5] for col in num_cols if 'round' not in col]
#     for col in num_cols:
#         try:
#             train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
#             test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
#         except:
#             pass
#     # Transform float64 and float32 to float16
#     num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
#     for col in tqdm(num_cols):
#         train[col] = train[col].astype(np.float16)
#         test[col] = test[col].astype(np.float16)
#     # Get feature list
#     features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    
# #     params = {
# #         'objective': 'binary:logistic', 
# #         'tree_method': 'hist', #gpu_hist #hist
# #         'max_depth': 7,
# #         'subsample':0.88,
# #         'colsample_bytree': 0.5,
# #         'gamma':1.5,
# #         'min_child_weight':8,
# #         'lambda':70,
# #         'eta':0.03}
    
#     # Create a numpy array to store test predictions
#     test_predictions = np.zeros(len(test))
#     # Create a numpy array to store out of folds predictions
#     oof_predictions = np.zeros(len(train))
#     kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
#     for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
#         print(' ')
#         print('-'*50)
#         print(f'Training fold {fold} with {len(features)} features...')
#         x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
#         y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
#         # lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
#         # lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        
#         val_pred , model = cat_train(x_train, y_train, x_val, y_val,cat_features)
        
#         model.save_model(f"../output/exp05 Cat lag feature/catboost_fold{fold}.cbm")
#         # Save best model
#         # joblib.dump(model, f'../output/exp04 XGB lag feature/{CFG.model}_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
#         # Predict validation
        
#         # val_pred = model.predict(x_val)
        
#         # Add to out of folds array
#         oof_predictions[val_ind] = val_pred
#         # Predict the test set
        
        
#         dtest = xgb.DMatrix(data=test[features])
#         test_pred = model.predict(dtest)
#         test_predictions += test_pred / CFG.n_folds
        
#         # Compute fold metric
#         score = amex_metric(y_val, val_pred)
#         print(f'Our fold {fold} CV score is {score}')
#         del x_train, x_val, y_train, y_val
#         gc.collect()
#     # Compute out of folds metric
#     score = amex_metric(train[CFG.target], oof_predictions)
#     print(f'Our out of folds CV score is {score}')
#     # Create a dataframe to store out of folds predictions
#     oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
#     oof_df.to_csv(f'../output/exp05 Cat lag feature/oof_{CFG.model}_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
#     # Create a dataframe to store test prediction
#     test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
#     test_df.to_csv(f'../output/exp05 Cat lag feature/test_{CFG.model}_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    


In [3]:

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '../input/amex-fe/'
    seed = 45
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'
    model = "cat"

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
    return train, test

# ====================================================
# XGB train
# ====================================================

def cat_train(x, y, xt, yt,cat_features):
    print("# of features:", x.shape[1])
    assert x.shape[1] == xt.shape[1]
    
    prams = {
        'depth': 8,
        'iterations':5000,
#         'learning_rate': 0.05,
        'random_state':CFG.seed,
    }

#     watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    watchlist = [(x, 'train'), (xt, 'eval')]
    clf = CatBoostClassifier(**prams)# 5000
    clf.fit(x, y, eval_set=[(xt, yt)], cat_features=cat_features,plot=True, verbose_eval = 100)
#     print('best ntree_limit:', clf.best_ntree_limit)
#     print('best score:', clf.best_score)
    # return clf.predict_proba(xt)[:, 1]
    return clf.predict_proba(xt)[:, 1], clf



# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


def xgb_amex(y_pred, y_true):
    return 'amex', amex_metric_np(y_pred,y_true.get_label())


# Created by https://www.kaggle.com/yunchonggan
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def amex_metric_np(preds: np.ndarray, target: np.ndarray) -> float:
    indices = np.argsort(preds)[::-1]
    preds, target = preds[indices], target[indices]

    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_mask = cum_norm_weight <= 0.04
    d = np.sum(target[four_pct_mask]) / np.sum(target)

    weighted_target = target * weight
    lorentz = (weighted_target / weighted_target.sum()).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    n_pos = np.sum(target)
    n_neg = target.shape[0] - n_pos
    gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)

    g = gini / gini_max
    return 0.5 * (g + d)



In [4]:
seed_everything(CFG.seed)
train, test = read_data()


In [5]:
# train_and_evaluate(train, test)

In [6]:



# ====================================================
# Train & Evaluate
# ====================================================
# def train_and_evaluate(train, test):
    # Label encode categorical features
    
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
cat_features = [f"{cf}_last" for cf in cat_features]
for cat_col in cat_features:
    encoder = LabelEncoder()
    train[cat_col] = encoder.fit_transform(train[cat_col])
    test[cat_col] = encoder.transform(test[cat_col])
# Round last float features to 2 decimal place
num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
num_cols = [col for col in num_cols if 'last' in col]
for col in num_cols:
    train[col + '_round2'] = train[col].round(2)
    test[col + '_round2'] = test[col].round(2)
# Get the difference between last and mean
num_cols = [col for col in train.columns if 'last' in col]
num_cols = [col[:-5] for col in num_cols if 'round' not in col]
for col in num_cols:
    try:
        train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
        test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
    except:
        pass
# Transform float64 and float32 to float16
num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
for col in tqdm(num_cols):
    train[col] = train[col].astype(np.float16)
    test[col] = test[col].astype(np.float16)
# Get feature list
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]

#     params = {
#         'objective': 'binary:logistic', 
#         'tree_method': 'hist', #gpu_hist #hist
#         'max_depth': 7,
#         'subsample':0.88,
#         'colsample_bytree': 0.5,
#         'gamma':1.5,
#         'min_child_weight':8,
#         'lambda':70,
#         'eta':0.03}

# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))

# cid = []
# cids = []

preds = []

kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
    # lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
    # lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)

    val_pred , model = cat_train(x_train, y_train, x_val, y_val,cat_features)

    model.save_model(f"../output/exp05 Cat lag feature/catboost_fold{fold}.cbm")
    # Save best model
    # joblib.dump(model, f'../output/exp04 XGB lag feature/{CFG.model}_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
    # Predict validation

    # val_pred = model.predict(x_val)

    # Add to out of folds array
    oof_predictions[val_ind] = val_pred
    # Predict the test set


#     dtest = xgb.DMatrix(data=test[features])
    pred = model.predict_proba(test[features])[:, 1]
    preds.append(pred)
    
#     test_predictions += test_pred / CFG.n_folds

    # Compute fold metric
    score = amex_metric(y_val, val_pred)
    print(f'Our fold {fold} CV score is {score}')
    del x_train, x_val, y_train, y_val, pred
    gc.collect()
# Compute out of folds metric
score = amex_metric(train[CFG.target], oof_predictions)
print(f'Our out of folds CV score is {score}')

test_predictions = np.mean(preds,axis = 0)

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(f'../output/exp05 Cat lag feature/oof_{CFG.model}_{score}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
# Create a dataframe to store test prediction
test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
test_df.to_csv(f'../output/exp05 Cat lag feature/test_{CFG.model}_{score}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


  0%|          | 0/903 [00:00<?, ?it/s]

 
--------------------------------------------------
Training fold 0 with 1188 features...
# of features: 1188


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.067666
0:	learn: 0.6065603	test: 0.6065922	best: 0.6065922 (0)	total: 902ms	remaining: 1h 15m 7s
100:	learn: 0.2213740	test: 0.2257566	best: 0.2257566 (100)	total: 1m 9s	remaining: 56m 10s
200:	learn: 0.2127444	test: 0.2215959	best: 0.2215959 (200)	total: 2m 15s	remaining: 53m 58s
300:	learn: 0.2057095	test: 0.2198187	best: 0.2198187 (300)	total: 3m 21s	remaining: 52m 19s
400:	learn: 0.1997982	test: 0.2188949	best: 0.2188949 (400)	total: 4m 26s	remaining: 50m 53s
500:	learn: 0.1945083	test: 0.2184726	best: 0.2184726 (500)	total: 5m 33s	remaining: 49m 52s
600:	learn: 0.1897354	test: 0.2182379	best: 0.2182345 (593)	total: 6m 37s	remaining: 48m 31s
700:	learn: 0.1852421	test: 0.2181327	best: 0.2181327 (694)	total: 7m 42s	remaining: 47m 13s
800:	learn: 0.1810323	test: 0.2181069	best: 0.2181069 (800)	total: 8m 46s	remaining: 45m 57s
900:	learn: 0.1769531	test: 0.2180664	best: 0.2180411 (832)	total: 9m 50s	remaining: 44m 47s
1000:	learn: 0.1728920	test: 0.2179553	best:

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

...

Learning rate set to 0.067666
0:	learn: 0.6096071	test: 0.6096955	best: 0.6096955 (0)	total: 757ms	remaining: 1h 3m 2s
100:	learn: 0.2216433	test: 0.2266338	best: 0.2266338 (100)	total: 1m 9s	remaining: 56m 30s
200:	learn: 0.2129568	test: 0.2219427	best: 0.2219427 (200)	total: 2m 17s	remaining: 54m 32s
300:	learn: 0.2057162	test: 0.2201414	best: 0.2201414 (300)	total: 3m 24s	remaining: 53m 6s
400:	learn: 0.1996964	test: 0.2191850	best: 0.2191850 (400)	total: 4m 30s	remaining: 51m 37s
500:	learn: 0.1946461	test: 0.2185865	best: 0.2185810 (499)	total: 5m 35s	remaining: 50m 11s
600:	learn: 0.1899713	test: 0.2182657	best: 0.2182657 (600)	total: 6m 40s	remaining: 48m 51s
700:	learn: 0.1855842	test: 0.2180708	best: 0.2180708 (700)	total: 7m 45s	remaining: 47m 33s
800:	learn: 0.1811299	test: 0.2178842	best: 0.2178825 (791)	total: 8m 50s	remaining: 46m 20s
900:	learn: 0.1771616	test: 0.2177575	best: 0.2177575 (900)	total: 9m 55s	remaining: 45m 7s
1000:	learn: 0.1731954	test: 0.2176790	best: 0.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.067666
0:	learn: 0.6062717	test: 0.6065356	best: 0.6065356 (0)	total: 664ms	remaining: 55m 20s
100:	learn: 0.2210687	test: 0.2280274	best: 0.2280274 (100)	total: 1m 9s	remaining: 56m 15s
200:	learn: 0.2122680	test: 0.2239263	best: 0.2239263 (200)	total: 2m 16s	remaining: 54m 27s
300:	learn: 0.2051685	test: 0.2223710	best: 0.2223659 (298)	total: 3m 22s	remaining: 52m 45s
400:	learn: 0.1993078	test: 0.2215360	best: 0.2215347 (399)	total: 4m 29s	remaining: 51m 25s
500:	learn: 0.1945325	test: 0.2210387	best: 0.2210297 (496)	total: 5m 33s	remaining: 49m 57s
600:	learn: 0.1902760	test: 0.2207943	best: 0.2207943 (600)	total: 6m 37s	remaining: 48m 31s
700:	learn: 0.1857507	test: 0.2204757	best: 0.2204757 (700)	total: 7m 43s	remaining: 47m 19s
800:	learn: 0.1817267	test: 0.2202418	best: 0.2202082 (791)	total: 8m 47s	remaining: 46m 3s
900:	learn: 0.1774941	test: 0.2201164	best: 0.2201164 (900)	total: 9m 51s	remaining: 44m 53s
1000:	learn: 0.1735979	test: 0.2201299	best: 0.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.067666
0:	learn: 0.6102061	test: 0.6104152	best: 0.6104152 (0)	total: 666ms	remaining: 55m 29s
100:	learn: 0.2215105	test: 0.2266697	best: 0.2266697 (100)	total: 1m 9s	remaining: 56m 29s
200:	learn: 0.2128081	test: 0.2223111	best: 0.2223111 (200)	total: 2m 17s	remaining: 54m 35s
300:	learn: 0.2058147	test: 0.2204938	best: 0.2204938 (300)	total: 3m 23s	remaining: 53m 1s
400:	learn: 0.2001478	test: 0.2195306	best: 0.2195292 (399)	total: 4m 29s	remaining: 51m 30s
500:	learn: 0.1950436	test: 0.2190489	best: 0.2190473 (490)	total: 5m 35s	remaining: 50m 10s
600:	learn: 0.1901630	test: 0.2187953	best: 0.2187953 (600)	total: 6m 40s	remaining: 48m 54s
700:	learn: 0.1857236	test: 0.2186276	best: 0.2186241 (698)	total: 7m 45s	remaining: 47m 34s
800:	learn: 0.1812505	test: 0.2184971	best: 0.2184933 (765)	total: 8m 51s	remaining: 46m 25s
900:	learn: 0.1771929	test: 0.2182983	best: 0.2182845 (893)	total: 9m 55s	remaining: 45m 10s
1000:	learn: 0.1733924	test: 0.2182743	best: 0.